In [2]:
from datasets import load_dataset

In [4]:
folder="my-data-chat/" 
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

In [5]:
def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

In [6]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")

In [7]:
dataset = dataset.shuffle().select(range(150))
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
dataset = dataset.train_test_split(test_size=50/150)

Map: 100%|██████████| 150/150 [00:00<00:00, 16870.79 examples/s]


In [8]:
dataset_test_valid = dataset['test'].train_test_split(0.5)
print(dataset["train"][45]["messages"])
dataset["train"].to_json(folder + "train.jsonl", orient="records")
dataset_test_valid["train"].to_json(folder + "test.jsonl", orient="records")
dataset_test_valid["test"].to_json(folder + "valid.jsonl", orient="records")

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_27871460_2 (network VARCHAR, state_or_territory VARCHAR)', 'role': 'system'}, {'content': 'Which network was located in Illinois?', 'role': 'user'}, {'content': 'SELECT network FROM table_27871460_2 WHERE state_or_territory = "Illinois"', 'role': 'assistant'}]


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1347.35ba/s]


11690

In [10]:
from mlx_lm.lora import  train

In [12]:
help(train)

Help on function train in module mlx_lm.tuner.trainer:

train(
    model,
    tokenizer,
    optimizer,
    train_dataset,
    val_dataset,
    args: mlx_lm.tuner.trainer.TrainingArgs = TrainingArgs(batch_size=4, iters=100, val_batches=25, steps_per_report=10, steps_per_eval=200, steps_per_save=100, max_seq_length=2048, adapter_file='adapters.safetensors', grad_checkpoint=False),
    loss: <built-in function callable> = <function default_loss at 0x147255300>,
    iterate_batches: <built-in function callable> = <function iterate_batches at 0x147255440>,
    training_callback: mlx_lm.tuner.trainer.TrainingCallback = None
)



In [11]:
# train(model='mistralai/Mistral-7B-Instruct-v0.2',train_dataset='my-data-chat/train.jsonl',val_dataset='my-data-chat/valid.jsonl' ,iterate_batches=2)

TypeError: train() missing 2 required positional arguments: 'tokenizer' and 'optimizer'

For the Fine tune the model need to crate the dataset according to model like for the mistral we can use the data 

{"messages":[{"content":"You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_97 (tie_no VARCHAR, away_team VARCHAR)","role":"system"},{"content":"How many ties does Walsall have?","role":"user"},{"content":"SELECT tie_no FROM table_name_97 WHERE away_team = \"walsall\"","role":"assistant"}]}

with jsonl file 



we have seprate file for the train.jsonl and valid.jsonl and test.jsonl

I am using the mlx libary to train my model this mlx libary supported by the apple silicaon chip 

For that we need to install the library 


pip install mlx_lm 


For the download the model using the command line 



mlx_lm.generate --model mistralai/Mistral-7B-Instruct-v0.2 --prompt "hello"

it will download model from the hugginface and store it in to locally 


For the training the we use the dataset we created 

and then we can run the traing using the command 

mlx_lm.lora --train --model mistralai/Mistral-7B-Instruct-v0.2 --data my-data-chat --batch-size 2


it will take some time and then it will train the model 

it will create the adapters folder in that we have weight for the model using this weight we can creat the ollam model and then use this model locally 


here is setups 

1. create the Modelfile 
    FROM mistral //model name
    ADAPTER ./adapters //adapter location 

2. run ollam command to create the model 

    ollama create <sqlmistral> -f Modelfile

3. run newly created model locally 
    ollama run sqlmistral


    



